Following is from the Twitter [thread](https://twitter.com/1st1/status/1160956397216866305) by Yuri Selivanov (@1st1).

#### Python Metaclasses

Let's talk about Python metaclasses. Many think they are magical things that mere mortals should never use. That's not quite true, although in many cases metaclasses can needlessly complicate things. Beware. 

Note: the purpose of this thread is to show how to use metaclasses via a few examples. There are a few simplifications here and there for the sake of brevity. If you want to understand how things work I suggest to start with the official documentation: [Link](https://docs.python.org/3/reference/datamodel.html#metaclasses)

1 - We first need to talk about regular classes, and specifically about `__new__()` and `__init__()` methods. The former is actually a static method-it's called when an instance of the class is constructed. The `__init__()` method then initializes that instance.

In [1]:
class Foo:
    def __new__(cls):
        print('about to construct an instance of', cls)
        obj = super().__new__(cls)
        print(obj, 'is a new instance of', cls)
        return obj
    
    def __init__(self):
        print('initializing', self)
      
Foo()    

about to construct an instance of <class '__main__.Foo'>
<__main__.Foo object at 0x035B59F0> is a new instance of <class '__main__.Foo'>
initializing <__main__.Foo object at 0x035B59F0>


2 - If we go low-level we will realize that the `class` statement is a syntax sugar. For example, the previous example is (pretty much) equivalent to this:


In [3]:
def Foo_new(cls):
    print('about to construct an instance of', cls)
    obj = object.__new__(cls)
    print(obj, 'is a new instance of', cls)
    return obj

def Foo_init(self):
    print('initializing', self)
    
Foo = type('Foo', (object,),{'__new__': Foo_new, '__init__': Foo_init})    

Foo()

about to construct an instance of <class '__main__.Foo'>
<__main__.Foo object at 0x035CDAD0> is a new instance of <class '__main__.Foo'>
initializing <__main__.Foo object at 0x035CDAD0>


3 - The `class` statement does a few more things than simply calling `type(...)`, but that's not important here. What matters is that we have just created a class via a function call! So there's nothing super special about the `class` syntax.

4 - Everything is an object in Python. Objects are instances of classes.

Classes are objects. Which means...that classes are instances of some classes. We call classes that create classes 'metaclasses'.

5 - A class is a metaclass when it's derived from the `type` class. To later use a metaclass, we use slightly unintuitive (at first) syntax:

```python

class FooMeta(type):
    """A metaclass for Foo."""
    
class Foo(metaclass = FooMeta):
    pass
```


6 - If you are really interested in how things work, now would be a good time to open a REPL and play with the snippet in (5). Specifically google about the `__mro__` attribute and try inspecting `FooMeta.__mro__` and `Foo.__mro__`. It's not that important right now, though.

In [4]:
class FooMeta(type):
    pass

class Foo(metaclass = FooMeta):
    pass

FooMeta.__mro__

(__main__.FooMeta, type, object)

In [5]:
Foo.__mro__

(__main__.Foo, object)

In [6]:
type(Foo)

__main__.FooMeta

7 - You only need to use `metaclass = FooMeta` syntax for the topmost class of the hiearchy. E.g. if you derive class `Bar` from `Foo` with the `class Bar(Foo)` syntax, the metaclass of `Bar` will be `FooMeta`. That's convenient. 

8 - Now let's get back to `__new__` and `__init__` methods we talked about in (1). What if we specify them for a metaclass? We certainly can. In that case they will allow us to fully control how classes are constructed:

In [7]:
class FooMeta(type):
    def __new__(mcls, name, bases, ns):
        print(
        'about to construct a class of', mcls, 'with name = ', name, 'with bases = ', bases, 'with ns = ', ns)
        
        cls = super().__new__(mcls, name, bases, ns)
        print('constructed a class', cls)
        return cls
    
    def __init__(cls, name, bases, ns):
        print('initializing', cls)
        
class Foo(metaclass = FooMeta):
    pass

class Bar(Foo):
    pass

about to construct a class of <class '__main__.FooMeta'> with name =  Foo with bases =  () with ns =  {'__module__': '__main__', '__qualname__': 'Foo'}
constructed a class <class '__main__.Foo'>
initializing <class '__main__.Foo'>
about to construct a class of <class '__main__.FooMeta'> with name =  Bar with bases =  (<class '__main__.Foo'>,) with ns =  {'__module__': '__main__', '__qualname__': 'Bar'}
constructed a class <class '__main__.Bar'>
initializing <class '__main__.Bar'>


9 - It's possible to change class bodies, class attibute and methods, names, list of base classes, and many other things via a metaclass with a `__new__()` method. People use metaclasses to automatically register classes, implement ORMs, enforce behaviours.

10 - Try writing some diabolical code by overloading operators, `__getitem__`, or `__prepare__` on metaclasses!

11 - There's a complication though. When metaclasses are used with multiple inheritance, it's easy to arrive to a situation whn there's more than one custom metaclass to chose from:

In [8]:
class FooMeta(type):
    pass

class BarMeta(type):
    pass

class Foo(metaclass = FooMeta):
    pass

class Bar(metaclass = BarMeta):
    pass

class Spam(Foo, Bar):
    pass





TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

12 - To solve (11), we added a new magic method to Python 3.6: `__init_subclass__`. It allows to implement many of the things people use metaclass for with a simpler pattern. E.g. here's a primitive class registry:

In [10]:
foos = []
class Foo:
    def __init_subclass__(cls):
        foos.append(cls)
        
class Bar(Foo):
    pass

class Spam(Foo):
    pass

print(foos)

[<class '__main__.Bar'>, <class '__main__.Spam'>]


There are multiple legitimate use cases for metaclasses. The only way to discover those use cases is by studying how frameworks use metaclasses and by experimenting on your own. I hope that I was able to demystify metaclasses somewhat in this thread. Go discover!

--**end of thread**--

- In Python eveything is an object. Every object has an `id`, a value and a `type`.
- Also, since everything is object,

 - we can assign it to a variable
 - we can pass it to a function
 - it can be returned by a function
 - we can add attributes to it
 - we can copy it
 
- Every object is either a class or an instance of a class. 
- A metaclass is the class of a class, i.e. a class is an instance of a metaclass. 
- Alternatively, every object in Python is an instance of a class. 
- `__bases__` attribute is used to find the classes an object inherits from. 
- `__class__` attribute is used to find the type of an object. 
- `__dict__` attribute is used to find all the attributes/methods of an object.

 
- In addition to checking the type of an object, `type` also has a completely  different ability. It can also create classes on the fly. `type` can take the description of a class as parameters, and return a class. 

In [17]:
object.__dict__

mappingproxy({'__class__': <attribute '__class__' of 'object' objects>,
              '__delattr__': <slot wrapper '__delattr__' of 'object' objects>,
              '__dir__': <method '__dir__' of 'object' objects>,
              '__doc__': 'The most base type',
              '__eq__': <slot wrapper '__eq__' of 'object' objects>,
              '__format__': <method '__format__' of 'object' objects>,
              '__ge__': <slot wrapper '__ge__' of 'object' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'object' objects>,
              '__gt__': <slot wrapper '__gt__' of 'object' objects>,
              '__hash__': <slot wrapper '__hash__' of 'object' objects>,
              '__init__': <slot wrapper '__init__' of 'object' objects>,
              '__init_subclass__': <method '__init_subclass__' of 'object' objects>,
              '__le__': <slot wrapper '__le__' of 'object' objects>,
              '__lt__': <slot wrapper '__lt__' of 'object' objects>,
 

In [24]:
print(type(object))
print(object.__bases__)
print(object.__class__)

<class 'type'>
()
<class 'type'>


In [25]:
class Test():
    pass

def echo(o):
    print(o)
    
print(Test)    

<class '__main__.Test'>


In [26]:
echo(Test)

<class '__main__.Test'>


In [27]:
hasattr(Test, 'some_attr')

False

In [28]:
Test.some_attr = 1
hasattr(Test, 'some_attr')

True

In [29]:
Test1 = Test
Test1.some_attr

1

In [19]:
a = 1
isinstance(a, int)

True

In [12]:
a = int()
print(a)
print(type(a))
print(type(int))
print(isinstance(a, int))

0
<class 'int'>
<class 'type'>
True


In [14]:
c = None
type(c), type(type(c))

(NoneType, type)

In [15]:
l = [1,2]
type(l), type(type(l))

(list, type)

In [5]:
int.__bases__, int.__class__

((object,), type)

In [8]:
b = float()
print(b)
print(type(b))

0.0
<class 'float'>


In [10]:
def test():
    pass

print(type(test))

<class 'function'>


In [11]:
print(type(type(test)))

<class 'type'>


In [2]:
class Foo():
    pass

foo = Foo()
print(type(foo))
print(foo.__class__)
print(type(Foo))
print(Foo.__class__)
print(Foo.__bases__)

<class '__main__.Foo'>
<class '__main__.Foo'>
<class 'type'>
<class 'type'>
(<class 'object'>,)


In [3]:
print(type(type))
print(type.__class__)
print(type.__class__.__class__)
print(type.__bases__)

<class 'type'>
<class 'type'>
<class 'type'>
(<class 'object'>,)


In [3]:
isinstance(foo, Foo), isinstance(Foo, type)

(True, True)

In [4]:
type.__bases__

(object,)

In [5]:
Foo.__bases__

(object,)

In [6]:
object.__bases__

()

Since `type` is in itself a class, we can subclass it to create a new `type`. Below we created a new `type` called `dd` by subclassing `type`. Now we can create classes from this newly created type `dd`. See below -

In [7]:
class dd(type):
    pass

class Complex(metaclass = dd):
    pass

In [8]:
print(type(Complex))
print(type(dd))

<class '__main__.dd'>
<class 'type'>


In [9]:
type(dd) is type(type)


True

In [10]:
isinstance(Complex, dd), isinstance(dd, type), isinstance(type, type), isinstance(Complex, type)

(True, True, True, True)

In [11]:
dd == type

False

In [12]:
Complex.__class__

__main__.dd

In [13]:
class Baz(metaclass = type):
    pass

In [14]:
Baz.__class__

type

When we created class `Foo` above, it implicitly became an instance of class `type`. However class `Complex` was created from `dd` type. See what happened in the case of `Baz` class.  

The fact that classes are instances of a class `type` allows us to program metaclasses. We can create classes, which inherit from the class `type`. So, a metaclass is a subclass of the class `type`.

We can subclass `type` by calling `type` with three parameters:

```
type(classname, superclasses, attributes_dict)
```

If type is called with three arguments, it will return a new type object. This provides us with a dynamic form of the class statement.

 - "classname" is a string defining the class name and becomes the name attribute;
 - "superclasses" is a list or tuple with the superclasses of our class. This list or tuple will become the bases attribute;
 - the attributes_dict is a dictionary, functioning as the namespace of our class. It contains the definitions for the class body and it becomes the dict attribute.


In [15]:
AA = type('AA',(),{})
AA

__main__.AA

In [16]:
x = AA()
x

In [17]:
type(AA), AA.__class__

(type, type)

In [18]:
class Robot:
    counter = 0
    def __init__(self, name):
        self.name = name
    def sayHello(self):
        return "Hi, I am " + self.name
    
def Rob_init(self, name):
    self.name = name

Robot2 = type("Robot2",(), {"counter":0, "__init__": Rob_init, "sayHello": lambda self: "Hi, I am " + self.name})

x = Robot2("Marvin")

print(x.name)
print(x.sayHello())

y = Robot("Marvin")

print(y.name)
print(y.sayHello())

print(x.__dict__)
print(y.__dict__)

Marvin
Hi, I am Marvin
Marvin
Hi, I am Marvin
{'name': 'Marvin'}
{'name': 'Marvin'}


In [24]:
dd = type(a)
dd

str

In [25]:
class dd(type):
    pass

class Complex(metaclass = dd):
    pass

In [26]:
ee = Complex()
ee

In [27]:
ee.__class__.__class__

__main__.dd

In [28]:
Complex.__class__

__main__.dd

In [29]:
print(type(Complex))

<class '__main__.dd'>


In [30]:
type(ee)

__main__.Complex

In [31]:
type(dd)

type

In [32]:
Foo

__main__.Foo

In [33]:
class ABC(Foo):
    pass

xyz = ABC()
xyz

In [34]:
isinstance(xyz, Foo)

True

In [35]:
a = 5
a.__class__

int

In [36]:
a.__class__.__class__

type

In [1]:
a =1
a.__class__

int

In [2]:
type(a).__name__

'int'

In [3]:
class Test:
    pass

print(Test.__class__)

<class 'type'>
